In [ ]:
# Install and load libraries

In [1]:
%conda install -c conda-forge lightgbm 

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::tqdm==4.62.3=pyhd8ed1ab_0
  - conda-forge/noarch::black==21.11b1=pyhd8ed1ab_0
  - conda-forge/linux-64::conda-package-handling==1.7.3=py38h497a2fe_1
  - conda-forge/noarch::dask-core==2021.11.2=pyhd8ed1ab_0
  - conda-forge/noarch::imageio==2.9.0=py_0
  - conda-forge/linux-64::pytest==6.2.5=py38h578d9bd_1
  - conda-forge/linux-64::watchdog==2.1.6=py38h578d9bd_1
  - conda-forge/linux-64::aiohttp==3.8.1=py38h497a2fe_0
  - conda-forge/linux-64::astropy==5.0=py38h6c62de6_0
  - conda-forge/linux-64::bokeh==2.4.2=py38h578d9bd_0
  - conda-forge/linux-64::distributed==2021.11.2=py38h578d9bd_0
  - conda-forge/noarch::flask==2.0.2=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==3.5.0=py38hf4fb855_0
  - conda-forge/noarch::nbformat==5.1.3=pyhd8ed1ab_0
  - conda-forge/noarch::pylint==2.12.1=pyhd8ed1ab_0
  - conda-forg

In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import glob
from lightgbm import LGBMRegressor
import random
from sklearn.model_selection import ParameterSampler
import scipy
import gc
from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm

In [2]:
np.random.seed(54321)
random.seed(54321)

# Read in Data and Prepare for Modeling

In [3]:
# Create an empty list to hold the dataframes of highways england data
total_df_list = list()

# Loop through the files, sorted in alphabetical order
# Read them into a df, make sure they are sorted by timestamp, and append to the list
for fname in sorted(glob.glob("Data/Unseen Sensor/Processed/*.csv")):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname) #, parse_dates=['timestamp'], index_col=['timestamp'])
    df = df.sort_values(by="timestamp")
    total_df_list.append(df)

Reading Data/Unseen Sensor/Processed/A19-9336-1_Northbound_2019_Processed.csv
Reading Data/Unseen Sensor/Processed/A66-9521-1_Westbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M40-7048-2_Southbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M62-2056A_Eastbound_Processed.csv


In [4]:
# Read in the start and end points csv, and subtract 1 to deal with index differences between R and python
start_end = pd.read_csv("unseen_sensor_start_end_points.csv")
start_end["start"] = start_end["start"] - 1
start_end["end"] = start_end["end"]

In [5]:
# Create an empty list to hold the subset data frames (those with only 12 weeks of data per highway)
subset_df_list = list()

In [6]:
# For each df in our original total df list
for idx, df in enumerate(total_df_list):
        
    # Filter the timeframe based on the start_end_points csv files
    subset_df = df.iloc[start_end.iloc[idx,1]:start_end.iloc[idx,2], ]\
    .reset_index(drop=True).reset_index(drop=False)\
    .rename(columns={"index":"rn"})
    
    # Create a new field called train_val_test to differentiate each set of data
    subset_df["train_val_test"] = np.where(subset_df["rn"]<(96*7*8),
                                           "train",
                                           np.where(subset_df["rn"]<(96*7*10),
                                                    "val",
                                                    "test"
                                                   )
                                       )
    
    # Append to list
    subset_df_list.append(subset_df)

In [7]:
# Create a list of df's with only fields we need

# Initialize empty list
model_df_list = list()

# For df in subset list
for df in subset_df_list:
       
    # Extract the timestamp, the volume, and the train_val_test assignment
    model_df = df[['timestamp', 'total_volume', "train_val_test"]]\
    .rename(columns={'timestamp':'start', 'total_volume':'target'})
    
    # Append this df to the new list
    model_df_list.append(model_df)

## Create Lag Emebedded Matrices for each TS

In [8]:
lag_n = 840

In [9]:
# # Lag embed the data frames and save to a list
lag_embed_df_list = list()

for df in model_df_list:
    # For each df in our list
    for n in range(1, (lag_n+1)):
        # For each lag level, up to 960
        # Create a new column called target-n
        name = f"target-{n}"
        # Save the target shifted n values into this colume
        df[name] = df['target'].shift(n)
    # Append to list
    lag_embed_df_list.append(df)

/tmp/ipykernel_8605/3230010149.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = df['target'].shift(n)


In [10]:
# Split the lag embedded list into train, val, and test lists

# First, initialize empty lists for each train, val, and test
train_df_list = list()
val_df_list = list()
test_df_list = list()

for i in range(len(lag_embed_df_list)):
    # For each df in our list
    df = lag_embed_df_list[i].copy()

    # Add a ts_index of i+1 to join with clustering data from R
    df['ts_index'] = i + 1
    
    # Subset into train, val, and test df's based on the train_val_test_field
    train_df = df.query("train_val_test == 'train'").copy()
    val_df = df.query("train_val_test=='val'").copy()
    test_df = df.query("train_val_test=='test'").copy()
    
    # Append to appropriate lists
    train_df_list.append(train_df)
    val_df_list.append(val_df)
    test_df_list.append(test_df)

In [11]:
# Concat all dfs from the lists together to create one full train, val, and test df
train_df_full = pd.concat(train_df_list)
val_df_full = pd.concat(val_df_list)
test_df_full = pd.concat(test_df_list)

In [12]:
# Drop unneeded columns
train_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
val_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
test_df_full.drop(columns=['start', 'train_val_test'], inplace=True)

In [13]:
# Append the training and validation data together for later use
train_val_df_full = train_df_full.append(val_df_full)

In [14]:
# Delete unused variables to free up memory
del train_df_list
del val_df_list 
del test_df_list
del lag_embed_df_list
del model_df_list
del subset_df_list

In [15]:
# Force garbage collection to free up memory
gc.collect()

0

In [16]:
unseen_sen_clust = pd.read_csv("Results/Unseen Sensor/clust_assign.csv")

In [17]:
unseen_sen_clust.head()

,Unnamed: 0,rand,catch22,tsfeat,dtw
0,1,3,1,2,2
1,2,2,1,2,1
2,3,1,1,2,1
3,4,4,1,1,1


# Full Model

In [18]:
# Save model to file to use later
filename = 'Results/Global/LightGBM Bayes/Full/model'
mod = joblib.load(filename)

In [19]:
res = pd.read_csv('Results/Global/LightGBM Bayes/Full/residual.csv')

In [20]:
# Function to compute test preds
def compute_lgbm_test_preds(mod, data, lag_n):
    """Function which takes in: a model, test data, and the lag embedding to use, and returns a df of forecasts"""

    # Initialize an empty data frame to store preds
    pred_df = pd.DataFrame()
    
    # Loop through each individual time series index in the data set
    for ts_idx in data.ts_index.unique():
        # Create the X matrix for each one
        X = data.query("ts_index==@ts_idx").iloc[:,1:(lag_n+1)].copy()

        # Forecast for that X matrix
        preds = mod.predict(X)
        
        # Save the results to a temp data frame
        pred_df_sub = pd.DataFrame({"ts_index": ts_idx, "test_preds": preds})
        
        # Append to primary data frame
        pred_df = pred_df.append(pred_df_sub)
    
    # Return df of all preds with corresponding ts_index column
    return pred_df

In [21]:
unseen_test_preds = compute_lgbm_test_preds(mod, test_df_full, lag_n)

In [22]:
# Function to compute performance metrics on test data
def compute_lgbm_test_perf(preds, data):
    """Function which takes inputs: a data frame of test predictions, and a test data df,
    and which returns a data frame of model performance"""
    
    # Create an empty list to store model performance
    perf_ls = list()
    
    # For each time series index in our data set
    for ts_idx in data.ts_index.unique():
        # Get the target (actual) for that index
        y_sub = data.query("ts_index==@ts_idx").iloc[:,0]
        # Extract the corresponding forecasts
        preds_sub = preds.query("ts_index==@ts_idx").test_preds
        
        # Compute rmse, mae, and the mean of the true target value for those preds
        rmse_sub = mean_squared_error(y_sub, preds_sub, squared=False)
        mae_sub = mean_absolute_error(y_sub, preds_sub)
        mean_sub = np.mean(y_sub)
        
        # Save those metrics to a dictionary
        pred_dict = {"rmse": rmse_sub, "mae": mae_sub, "mean": mean_sub}
        
        # Append the dictionary to the list
        perf_ls.append(pred_dict)
        
    # Return a data frame of model performance created from the list of dictionaries
    return pd.DataFrame(perf_ls)

In [23]:
# Compute model perf metrics using above function
full_mod_test_perf = compute_lgbm_test_perf(unseen_test_preds, test_df_full)

# Compute scaled performance metrics in new columns
full_mod_test_perf['nrmse'] = full_mod_test_perf['rmse']/full_mod_test_perf['mean']
full_mod_test_perf['smae'] = full_mod_test_perf['mae']/full_mod_test_perf['mean']

In [24]:
# Print the means of model perf metrics
full_mod_test_perf.mean()

rmse      41.851829
mae       27.121187
mean     377.001860
nrmse      0.120557
smae       0.082307
dtype: float64

In [25]:
# Function to compute pred intervals with bootstrap method
def compute_lgbm_boostrap_int(preds, resid, n_boot):
    """Function which takes in a model's predictions and residuals, and a number of bootstrap resamples to use,
    and which outputs a df with pred intervals at 80% and 95%"""
    
    # Set seeds for reproducibility
    random.seed(54321)
    np.random.seed(54321)
    
    # Create empty columns in the pred df to store the PIs
    preds['lo_95'] = np.nan
    preds['hi_95'] = np.nan
    preds['lo_80'] = np.nan
    preds['hi_80'] = np.nan
    
    # For each row in the pred df
    for n in range(preds.shape[0]):
        # Sample with replacement n_boot times from the residuals
        resid_boot = np.random.choice(resid, size=n_boot, replace=True)
        # Extract the forecast value for that row
        pred_n = preds.iloc[n, :].test_preds
        # Add the residual vector to the forecast value
        pred_n_boot = resid_boot + pred_n
        
        # Compute quantiles of this residual+forecast vector
        percent_95_lo = np.percentile(pred_n_boot, 2.5)
        percent_95_hi = np.percentile(pred_n_boot, 97.5)
        
        percent_80_lo = np.percentile(pred_n_boot, 10)
        percent_80_hi = np.percentile(pred_n_boot, 90)
        
        # Save these quantiles to the appropriate df column
        preds.iloc[n, 2] = percent_95_lo
        preds.iloc[n, 3] = percent_95_hi
        preds.iloc[n, 4] = percent_80_lo
        preds.iloc[n, 5] = percent_80_hi
    
    # Return the updated preds data frame
    return preds

In [26]:
# Compute PIs with 1000 bootstrap samples
nboot = 1000
full_mod_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds, 
                                               res.residual.values, 
                                               nboot)

In [27]:
# Add the true values into their own df column
full_mod_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [28]:
full_mod_boot_ints.head()

,ts_index,test_preds,lo_95,hi_95,lo_80,hi_80,actual
0,1,305.268213,254.766238,364.341719,281.346342,334.368586,263.0
1,1,255.700589,199.820108,317.763844,225.932045,283.377616,242.0
2,1,223.922536,166.159973,288.504607,193.912266,250.418422,234.0
3,1,211.703889,151.296535,271.999550,184.541507,240.285101,188.0
4,1,179.286284,118.957231,237.056086,148.150057,207.128656,170.0


In [29]:
# Create a function to compute the interval score
def interval_score(true_values, lower, upper, interval_range):
    """ Function which takes in the true values, the upper and lower bounds of PIs, and the PI level (e.g., 90%)
        and from these inputs, computes the interval score for each prediction
    """
    
    # Compute alpha from the interval range
    alpha = 1-interval_range
    
    # Save the upper, lower, and true_values as numpy arrays for computation purposes
    upper = np.array(upper)
    lower = np.array(lower)
    true_values = np.array(true_values)
    
    # Compute the lower component of the interval score - just a boolean for true below interval
    def lower_ind(true,low):
        if true<low:
            return 1
        else:
            return 0
        
    # Computer the upper component of the interval score - similar boolean for true above interval
    def upper_ind(true,up):
        if true>up:
            return 1
        else:
            return 0
        
    # Computer the actual score for each obsveration - formula here: https://epiforecasts.io/scoringutils/reference/interval_score.html
    scores = (upper-lower) + (2/alpha)*(lower-true_values)*(lower > true_values) + (2/alpha)*(true_values-upper)*(true_values > upper)
    
    # Return the scores array
    return scores

In [30]:
# Compute the 95% and 80% PI scores using the above function as new data frame columns
full_mod_boot_ints['int_95_score'] = interval_score(full_mod_boot_ints.actual, 
                                                    full_mod_boot_ints.lo_95,
                                                    full_mod_boot_ints.hi_95,
                                                    0.95)
                                                    
full_mod_boot_ints['int_80_score'] = interval_score(full_mod_boot_ints.actual, 
                                                    full_mod_boot_ints.lo_80,
                                                    full_mod_boot_ints.hi_80,
                                                    0.80)

In [31]:
# Print the means of the interval scores
full_mod_boot_ints.mean()

ts_index          2.500000
test_preds      377.200035
lo_95           316.589974
hi_95           440.030265
lo_80           348.566140
hi_80           406.796210
actual          377.001860
int_95_score    373.881683
int_80_score    174.792619
dtype: float64

In [119]:
full_mod_boot_ints_group = full_mod_boot_ints.groupby("ts_index").mean().reset_index()
full_mod_boot_ints_group['mean'] = full_mod_test_perf['mean'].values
full_mod_boot_ints_group['int_95_score_scaled'] = full_mod_boot_ints_group['int_95_score']/full_mod_boot_ints_group['mean']
full_mod_boot_ints_group['int_80_score_scaled'] = full_mod_boot_ints_group['int_80_score']/full_mod_boot_ints_group['mean']

In [120]:
full_mod_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.977406
int_80_score_scaled    0.504571
dtype: float64

# Random Clusters

In [32]:
unseen_sen_clust.rand

0    3
1    2
2    1
3    4
Name: rand, dtype: int64

In [34]:
rand_clust_mod3 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_2")
rand_clust_mod2 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_1")
rand_clust_mod1 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_0")
rand_clust_mod4 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_3")

In [35]:
rand_clust_res = pd.read_csv("Results/Global/LightGBM Bayes/Random Cluster/residual.csv")
rand_clust_res['residual'] = rand_clust_res['residual'].apply(eval)

res_rand_clust_3 = rand_clust_res.query("cluster==3")['residual'].values[0]
res_rand_clust_2 = rand_clust_res.query("cluster==2")['residual'].values[0]
res_rand_clust_1 = rand_clust_res.query("cluster==1")['residual'].values[0]
res_rand_clust_4 = rand_clust_res.query("cluster==4")['residual'].values[0]

In [36]:
unseen_test_preds_rand_clust3 = compute_lgbm_test_preds(rand_clust_mod3, 
                                                        test_df_full.query("ts_index==1"), 
                                                        lag_n)

unseen_test_preds_rand_clust2 = compute_lgbm_test_preds(rand_clust_mod2, 
                                                        test_df_full.query("ts_index==2"), 
                                                        lag_n)

unseen_test_preds_rand_clust1 = compute_lgbm_test_preds(rand_clust_mod1, 
                                                        test_df_full.query("ts_index==3"), 
                                                        lag_n)

unseen_test_preds_rand_clust4 = compute_lgbm_test_preds(rand_clust_mod4, 
                                                        test_df_full.query("ts_index==4"), 
                                                        lag_n)

In [37]:
# Compute model perf metrics using above function
rand_clust3_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust3, test_df_full.query("ts_index==1"))
rand_clust2_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust2, test_df_full.query("ts_index==2"))
rand_clust1_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust1, test_df_full.query("ts_index==3"))
rand_clust4_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust4, test_df_full.query("ts_index==4"))

rand_clust_test_perf = rand_clust3_test_perf.append(rand_clust2_test_perf).append(rand_clust1_test_perf).append(rand_clust4_test_perf)

# Compute scaled performance metrics in new columns
rand_clust_test_perf['nrmse'] = rand_clust_test_perf['rmse']/rand_clust_test_perf['mean']
rand_clust_test_perf['smae'] = rand_clust_test_perf['mae']/rand_clust_test_perf['mean']

In [38]:
rand_clust_test_perf.mean()

rmse      43.554114
mae       29.037834
mean     377.001860
nrmse      0.126193
smae       0.087433
dtype: float64

In [39]:
# Compute PIs with 1000 bootstrap samples
rand3_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust3,
                                            res_rand_clust_3, 
                                            nboot)


rand2_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust2,
                                            res_rand_clust_2, 
                                            nboot)

rand1_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust1,
                                            res_rand_clust_1, 
                                            nboot)

rand4_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust4,
                                            res_rand_clust_4, 
                                            nboot)

In [40]:
rand_boot_ints = rand3_boot_ints.append(rand2_boot_ints).append(rand1_boot_ints).append(rand4_boot_ints)

In [41]:
# Add the true values into their own df column
rand_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [42]:
# Compute the 95% and 80% PI scores using the above function as new data frame columns
rand_boot_ints['int_95_score'] = interval_score(rand_boot_ints.actual, 
                                                    rand_boot_ints.lo_95,
                                                    rand_boot_ints.hi_95,
                                                    0.95)
                                                    
rand_boot_ints['int_80_score'] = interval_score(rand_boot_ints.actual, 
                                                    rand_boot_ints.lo_80,
                                                    rand_boot_ints.hi_80,
                                                    0.80)

In [43]:
rand_boot_ints.mean()

ts_index          2.500000
test_preds      374.756530
lo_95           328.684062
hi_95           422.503904
lo_80           350.424005
hi_80           399.815557
actual          377.001860
int_95_score    500.655999
int_80_score    198.306081
dtype: float64

In [117]:
rand_boot_ints_group = rand_boot_ints.groupby("ts_index").mean().reset_index()
rand_boot_ints_group['mean'] = rand_clust_test_perf['mean'].values
rand_boot_ints_group['int_95_score_scaled'] = rand_boot_ints_group['int_95_score']/rand_boot_ints_group['mean']
rand_boot_ints_group['int_80_score_scaled'] = rand_boot_ints_group['int_80_score']/rand_boot_ints_group['mean']

In [118]:
rand_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    1.102840
int_80_score_scaled    0.535848
dtype: float64

# Highway System

In [44]:
eng_mod = joblib.load("Results/Global/LightGBM Bayes/Highway System/model_0")

In [45]:
highway_res = pd.read_csv("Results/Global/LightGBM Bayes/Highway System/residual.csv")
highway_res['residual'] = highway_res['residual'].apply(eval)

eng_res = highway_res.query("cluster==1")['residual'].values[0]

In [46]:
unseen_test_preds_highway = compute_lgbm_test_preds(eng_mod, 
                                                    test_df_full,
                                                    lag_n)

In [47]:
highway_test_perf = compute_lgbm_test_perf(unseen_test_preds_highway, test_df_full)


In [48]:
highway_test_perf['nrmse'] = highway_test_perf['rmse']/highway_test_perf['mean']
highway_test_perf['smae'] = highway_test_perf['mae']/highway_test_perf['mean']

In [49]:
highway_test_perf.mean()

rmse      42.395630
mae       27.745831
mean     377.001860
nrmse      0.122915
smae       0.084693
dtype: float64

In [50]:
highway_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_highway,
                                              eng_res,
                                              nboot)

In [51]:
highway_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [52]:
highway_boot_ints['int_95_score'] = interval_score(highway_boot_ints.actual, 
                                                    highway_boot_ints.lo_95,
                                                    highway_boot_ints.hi_95,
                                                    0.95)
                                                    
highway_boot_ints['int_80_score'] = interval_score(highway_boot_ints.actual, 
                                                    highway_boot_ints.lo_80,
                                                    highway_boot_ints.hi_80,
                                                    0.80)

In [53]:
highway_boot_ints.mean()

ts_index          2.500000
test_preds      377.409187
lo_95           309.365376
hi_95           449.023181
lo_80           343.296052
hi_80           412.867392
actual          377.001860
int_95_score    363.019542
int_80_score    174.773428
dtype: float64

In [115]:
highway_boot_ints_group = highway_boot_ints.groupby("ts_index").mean().reset_index()
highway_boot_ints_group['mean'] = highway_test_perf['mean'].values
highway_boot_ints_group['int_95_score_scaled'] = highway_boot_ints_group['int_95_score']/highway_boot_ints_group['mean']
highway_boot_ints_group['int_80_score_scaled'] = highway_boot_ints_group['int_80_score']/highway_boot_ints_group['mean']

In [116]:
highway_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    1.013325
int_80_score_scaled    0.531538
dtype: float64

# Catch22 KMeans Clusters

In [54]:
unseen_sen_clust.catch22

0    1
1    1
2    1
3    1
Name: catch22, dtype: int64

In [55]:
catch22_mod1 = joblib.load("Results/Global/LightGBM Bayes/Catch22 KMeans/model_0")

In [56]:
catch22_res = pd.read_csv("Results/Global/LightGBM Bayes/Catch22 KMeans/residual.csv")
catch22_res['residual'] = catch22_res['residual'].apply(eval)

catch22_res1 = catch22_res.query("cluster==1")['residual'].values[0]

In [57]:
unseen_test_preds_catch22 = compute_lgbm_test_preds(catch22_mod1, 
                                                    test_df_full,
                                                    lag_n)

In [58]:
catch22_test_perf = compute_lgbm_test_perf(unseen_test_preds_catch22, test_df_full)


In [59]:
catch22_test_perf['nrmse'] = catch22_test_perf['rmse']/catch22_test_perf['mean']
catch22_test_perf['smae'] = catch22_test_perf['mae']/catch22_test_perf['mean']

In [60]:
catch22_test_perf.mean()

rmse      42.264104
mae       27.956650
mean     377.001860
nrmse      0.121633
smae       0.084240
dtype: float64

In [61]:
catch22_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_catch22,
                                               catch22_res1,
                                               nboot)

In [62]:
catch22_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [63]:
catch22_boot_ints['int_95_score'] = interval_score(catch22_boot_ints.actual, 
                                                    catch22_boot_ints.lo_95,
                                                    catch22_boot_ints.hi_95,
                                                    0.95)
                                                    
catch22_boot_ints['int_80_score'] = interval_score(catch22_boot_ints.actual, 
                                                    catch22_boot_ints.lo_80,
                                                    catch22_boot_ints.hi_80,
                                                    0.80)

In [64]:
catch22_boot_ints.mean()

ts_index          2.500000
test_preds      376.880176
lo_95           325.234952
hi_95           430.400636
lo_80           348.812294
hi_80           405.534113
actual          377.001860
int_95_score    407.694767
int_80_score    180.634133
dtype: float64

In [106]:
catch22_boot_ints_group = catch22_boot_ints.groupby("ts_index").mean().reset_index()
catch22_boot_ints_group['mean'] = catch22_test_perf['mean'].values
catch22_boot_ints_group['int_95_score_scaled'] = catch22_boot_ints_group['int_95_score']/catch22_boot_ints_group['mean']
catch22_boot_ints_group['int_80_score_scaled'] = catch22_boot_ints_group['int_80_score']/catch22_boot_ints_group['mean']

In [107]:
catch22_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.972671
int_80_score_scaled    0.509896
dtype: float64

# TSFeat KMeans Clusters

In [65]:
unseen_sen_clust.tsfeat

0    2
1    2
2    2
3    1
Name: tsfeat, dtype: int64

In [66]:
tsfeat_mod2 = joblib.load("Results/Global/LightGBM Bayes/TSFeat KMeans/model_1")
tsfeat_mod1 = joblib.load("Results/Global/LightGBM Bayes/TSFeat KMeans/model_0")

In [67]:
tsfeat_res = pd.read_csv("Results/Global/LightGBM Bayes/TSFeat KMeans/residual.csv")
tsfeat_res['residual'] = tsfeat_res['residual'].apply(eval)

tsfeat_res2 = tsfeat_res.query("cluster==2")['residual'].values[0]
tsfeat_res1 = tsfeat_res.query("cluster==1")['residual'].values[0]

In [70]:
unseen_test_preds_tsfeat2 = compute_lgbm_test_preds(tsfeat_mod2, 
                                                    test_df_full.query("ts_index!=4"),
                                                    lag_n)

unseen_test_preds_tsfeat1 = compute_lgbm_test_preds(tsfeat_mod1, 
                                                    test_df_full.query("ts_index==4"),
                                                    lag_n)

In [72]:
tsfeat_test_perf2 = compute_lgbm_test_perf(unseen_test_preds_tsfeat2, test_df_full.query("ts_index!=4"))
tsfeat_test_perf1 = compute_lgbm_test_perf(unseen_test_preds_tsfeat1, test_df_full.query("ts_index==4"))

tsfeat_test_perf = tsfeat_test_perf2.append(tsfeat_test_perf1)

In [73]:
tsfeat_test_perf['nrmse'] = tsfeat_test_perf['rmse']/tsfeat_test_perf['mean']
tsfeat_test_perf['smae'] = tsfeat_test_perf['mae']/tsfeat_test_perf['mean']

In [74]:
tsfeat_test_perf.mean()

rmse      42.200990
mae       28.080535
mean     377.001860
nrmse      0.121487
smae       0.083932
dtype: float64

In [75]:
tsfeat_boot_ints2 = compute_lgbm_boostrap_int(unseen_test_preds_tsfeat2,
                                              tsfeat_res2,
                                              nboot)

tsfeat_boot_ints1 = compute_lgbm_boostrap_int(unseen_test_preds_tsfeat1,
                                              tsfeat_res1,
                                              nboot)

tsfeat_boot_ints = tsfeat_boot_ints2.append(tsfeat_boot_ints1)

In [76]:
tsfeat_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [77]:
tsfeat_boot_ints['int_95_score'] = interval_score(tsfeat_boot_ints.actual, 
                                                    tsfeat_boot_ints.lo_95,
                                                    tsfeat_boot_ints.hi_95,
                                                    0.95)
                                                    
tsfeat_boot_ints['int_80_score'] = interval_score(tsfeat_boot_ints.actual, 
                                                    tsfeat_boot_ints.lo_80,
                                                    tsfeat_boot_ints.hi_80,
                                                    0.80)

In [78]:
tsfeat_boot_ints.mean()

ts_index          2.500000
test_preds      376.273824
lo_95           318.735205
hi_95           435.248455
lo_80           347.461421
hi_80           405.873810
actual          377.001860
int_95_score    386.593020
int_80_score    180.505476
dtype: float64

In [101]:
tsfeat_boot_ints_group = tsfeat_boot_ints.groupby("ts_index").mean().reset_index()
tsfeat_boot_ints_group['mean'] = tsfeat_test_perf['mean'].values
tsfeat_boot_ints_group['int_95_score_scaled'] = tsfeat_boot_ints_group['int_95_score']/tsfeat_boot_ints_group['mean']
tsfeat_boot_ints_group['int_80_score_scaled'] = tsfeat_boot_ints_group['int_80_score']/tsfeat_boot_ints_group['mean']

In [104]:
tsfeat_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.965833
int_80_score_scaled    0.509641
dtype: float64

# Train and Test - DTW Clusters

In [79]:
unseen_sen_clust.dtw

0    2
1    1
2    1
3    1
Name: dtw, dtype: int64

In [80]:
dtw_mod2 = joblib.load("Results/Global/LightGBM Bayes/DTW/model_1")
dtw_mod1 = joblib.load("Results/Global/LightGBM Bayes/DTW/model_0")

In [81]:
dtw_res = pd.read_csv("Results/Global/LightGBM Bayes/DTW/residual.csv")
dtw_res['residual'] = dtw_res['residual'].apply(eval)

dtw_res2 = dtw_res.query("cluster==2")['residual'].values[0]
dtw_res1 = dtw_res.query("cluster==1")['residual'].values[0]

In [82]:
unseen_test_preds_dtw2 = compute_lgbm_test_preds(dtw_mod2, 
                                                 test_df_full.query("ts_index==1"),
                                                 lag_n)

unseen_test_preds_dtw1 = compute_lgbm_test_preds(dtw_mod1, 
                                                 test_df_full.query("ts_index!=1"),
                                                 lag_n)

In [83]:
dtw_test_perf2 = compute_lgbm_test_perf(unseen_test_preds_dtw2, test_df_full.query("ts_index==1"))

dtw_test_perf1 = compute_lgbm_test_perf(unseen_test_preds_dtw1, test_df_full.query("ts_index!=1"))

dtw_test_perf = dtw_test_perf2.append(dtw_test_perf1)

In [84]:
dtw_test_perf['nrmse'] = dtw_test_perf['rmse']/dtw_test_perf['mean']
dtw_test_perf['smae'] = dtw_test_perf['mae']/dtw_test_perf['mean']

In [85]:
dtw_test_perf.mean()

rmse      42.581503
mae       28.182318
mean     377.001860
nrmse      0.129566
smae       0.092134
dtype: float64

In [86]:
dtw_boot_ints2 = compute_lgbm_boostrap_int(unseen_test_preds_dtw2,
                                           dtw_res2,
                                           nboot)

dtw_boot_ints1 = compute_lgbm_boostrap_int(unseen_test_preds_dtw1,
                                           dtw_res1,
                                           nboot)

In [87]:
dtw_boot_ints = dtw_boot_ints2.append(dtw_boot_ints1)
dtw_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [88]:
dtw_boot_ints['int_95_score'] = interval_score(dtw_boot_ints.actual, 
                                               dtw_boot_ints.lo_95,
                                               dtw_boot_ints.hi_95,
                                               0.95)
                                                    
dtw_boot_ints['int_80_score'] = interval_score(dtw_boot_ints.actual, 
                                               dtw_boot_ints.lo_80,
                                               dtw_boot_ints.hi_80,
                                               0.80)

In [89]:
dtw_boot_ints.mean()

ts_index          2.500000
test_preds      376.809457
lo_95           306.006473
hi_95           450.046898
lo_80           338.793128
hi_80           415.929342
actual          377.001860
int_95_score    334.566780
int_80_score    169.128788
dtype: float64

In [98]:
dtw_boot_ints_group = dtw_boot_ints.groupby("ts_index").mean().reset_index()
dtw_boot_ints_group['mean'] = dtw_test_perf['mean'].values
dtw_boot_ints_group['int_95_score_scaled'] = dtw_boot_ints_group['int_95_score']/dtw_boot_ints_group['mean']
dtw_boot_ints_group['int_80_score_scaled'] = dtw_boot_ints_group['int_80_score']/dtw_boot_ints_group['mean']

In [100]:
dtw_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    1.016889
int_80_score_scaled    0.561006
dtype: float64